In [2]:
from hoshi_workflow import make_initial_models as mim
import importlib
importlib.reload(mim)
mim.enable_logging()
import numpy as np
from datetime import datetime

In [4]:
path = "./example_model/fake_model"
hm = mim.HoshiModelDir(path)
print(hm.henyey_params)


{'input_structure_dir': 'strdata', 'output_summary_dir': 'summary', 'input_structure_file': 'M11F2d00_Z00F0d00', 'output_structure_file': 'M11F2d00_Z00F0d00', 'NAME_IN': 'e', 'NAME_OUT': '0', 'NSTG_max': '10000', 'MAXQTY': '1.d10\t2', 'FLAG_reduce_D_and_Li': '1', 'relative_metallicity': '0.0', 'FLAG_change_mass': '1', 'new_mass': '20.0'}


In [ ]:
# to execute 'evol' in the model directory, but here fake model is not able to run evol 
# one should test with real model directory
#hm.run()

In [8]:
new_henyey_params = {
    # 'input_structure_dir': 'strdata',
    # 'output_summary_dir': 'summary',
    'input_structure_file': 'M11F1d50_Z05F1d00',
    'output_structure_file': 'test',
    'NAME_IN': 'e',
    'NAME_OUT': '0',
    'NSTG_max': '1234',
    'MAXQTY': '1.d10\t2',
    'FLAG_reduce_D_and_Li': '1',
    'relative_metallicity': '5.d-6',
    'FLAG_change_mass': '1',
    'new_mass': '2.45d1',
}

hm.modify_input_henyey(new_params=new_henyey_params, dry_run=True)


IndexError: list index out of range